EDA on Used Cars Prices Dataset

Dataset: 
https://www.kaggle.com/datasets/austinreese/craigslist-carstrucks-data

In [27]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import kagglehub
import seaborn as sns

# Make plots look cleaner
sns.set()
from sklearn.preprocessing import LabelEncoder


In [28]:
path = kagglehub.dataset_download("austinreese/craigslist-carstrucks-data")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\N\.cache\kagglehub\datasets\austinreese\craigslist-carstrucks-data\versions\10


In [ ]:
cars_path = os.path.join(path, 'vehicles.csv') # dataframe path
df_cars = pd.read_csv(cars_path)

print(f"Dataset shape: {df_cars.shape}") #  (426880 rows, 26 columns)
df_cars.head()

In [ ]:
EDA:

In [ ]:
# Check data types and structure
df_cars.info()

In [ ]:
missing_percentage = (df_cars.isnull().sum() / len(df_cars)) * 100
missing_data = pd.DataFrame({
    'Column': missing_percentage.index,
    'Missing_Percentage': missing_percentage.values
})
missing_data = missing_data[missing_data['Missing_Percentage'] > 0].sort_values('Missing_Percentage', ascending=False)

print("Missing Data Analysis:")
missing_data.head(10)

In [ ]:
# Descriptive statistics for numerical columns
df_cars.describe()

In [ ]:
plt.figure(figsize=(10, 5))
plt.hist(df_cars['price'].dropna(), bins=50, edgecolor='black')
# must dropna in hist, bc NaN cannot go into bins, causes issues or wrong counts
plt.title('Price Distribution')
plt.xlabel('Price')
plt.ylabel('Frequency')
plt.show()

Price is heavily skewed near zero to 100,000 - many unrealistic listings.

In [ ]:
# Condition distribution
condition_counts = df_cars['condition'].value_counts()
#value_counts() returns: index = unique categories, values = their frequencies

plt.figure(figsize=(10, 5))
plt.bar(condition_counts.index, condition_counts.values, color='coral') # plt.bar(x axis, y axis)
# no dropna in bargraph bc is handels it automatically

plt.title('Condition Distribution')
plt.xlabel('Condition')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.show()

"good" condition has the most listings, followed by "Excellent"

In [ ]:
# Transmission distribution
transmission_counts = df_cars['transmission'].value_counts()
plt.figure(figsize=(10, 5))
plt.bar(transmission_counts.index, transmission_counts.values, color='teal')
plt.title('Transmission Distribution')
plt.xlabel('Transmission Type')
plt.ylabel('Count')
plt.xticks(rotation=45) # rotate the x-axis label 45 degrees
plt.show()

"Automatic" transmission is dominant with ~330k listings.

In [ ]:
# Year distribution
plt.figure(figsize=(10, 5))
plt.hist(df_cars['year'].dropna(), bins=30, edgecolor='black', color='orange')
plt.title('Year Distribution')
plt.xlabel('Year')
plt.ylabel('Frequency')
plt.show()

mostly new cars

In [ ]:
# Odometer distribution
plt.figure(figsize=(10, 5))
plt.hist(df_cars['odometer'].dropna(), bins=50, edgecolor='black', color='green')
plt.title('Odometer Distribution')
plt.xlabel('Odometer')
plt.ylabel('Frequency')
plt.show()

In [ ]:
# Top manufacturers
top_manufacturers = df_cars['manufacturer'].value_counts().head(10)
plt.figure(figsize=(10, 5))
plt.barh(top_manufacturers.index, top_manufacturers.values, color='purple')
# plt.barh() draws a horizontal bar chart. Bars go left to right,
# Categories on the y-axis, Values on the x-axis. used when we have mane labels

plt.title('Top 10 Manufacturers')
plt.xlabel('Count')
plt.gca().invert_yaxis() # largest value appears at the top, not the bottom.
plt.show()

In [ ]:
plt.figure(figsize=(6,4))
sns.heatmap(df_cars[['year','price']].corr(), annot=True)
plt.title("Correlation Matrix")
plt.show()

Data Cleaning

In [ ]:
# Select relevant columns, we do not select 'model' (too many unique values, too sparse and will hurt the model performance)
cols = ['price', 'year', 'manufacturer', 'condition', 'cylinders', 'fuel', 'odometer', 'transmission', 'type']
df_clean = df_cars[cols].copy()

# Drop rows where target (price) or key features are missing - can't predict without them
print(f"Before: {df_clean.shape}")
df_clean = df_clean.dropna(subset=['price', 'year', 'odometer']) # drop rows with these features misssing
print(f"After dropping missing price/year/odometer: {df_clean.shape}")

In [ ]:
# Fill categorical columns with 'unknown' - missing likely means "not specified"
for col in ['manufacturer', 'condition', 'fuel', 'transmission', 'type']:
    df_clean[col] = df_clean[col].fillna('unknown')

# Fill cylinders with mode - discrete feature, mode is most representative, instead of unknown
df_clean['cylinders'] = df_clean['cylinders'].fillna(df_clean['cylinders'].mode()[0])

print("Missing values remaining:", df_clean.isnull().sum().sum()) # first sum for each column, second for total sum

Encode Categorical Variables

In [ ]:
categorical_cols = ['manufacturer', 'condition', 'cylinders', 'fuel', 'transmission', 'type']
for col in categorical_cols:
    le = LabelEncoder()
    df_clean[col] = le.fit_transform(df_clean[col].astype(str))
    # astype(str) bc labelencoder takes string only, we need to ensure no error occur bc of numerical or nan values

df_clean.head()

Feature Engineering

In [ ]:
# Car age: older cars typically worth less
df_clean['age'] = 2026 - df_clean['year']

# Mileage per year: high usage = more wear
df_clean['miles_per_year'] = df_clean['odometer'] / (df_clean['age'])


# Interaction: captures combined effect of age and annual mileage
df_clean['age_x_miles_per_year'] = df_clean['miles_per_year'] * df_clean['age']

# Quick preview of engineered features
df_clean[['year', 'age', 'odometer', 'miles_per_year', 'age_x_miles_per_year']].head()